In [1]:
import configparser
from vkstatsbot.vk_api import VKApiConnector
import pickle
import pandas as pd

#reading config
config = configparser.ConfigParser()
config.read('vkstatsbot/vkstat.cfg')
vk_api_v = config.get('VKApi', "v")
vk_api_token = config.get('VKApi', 'token')
client_id = config.get('VKApi', 'client_id')

VKApiConnector.config(vk_api_v, client_id, vk_api_token)

In [ ]:
https://oauth.vk.com/authorize?client_id=6933695&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=offline,wall,friends&response_type=token&v=5.52

In [ ]:
https://oauth.vk.com/blank.html#
    access_token=c2c771bc01618a6cc828785aee9c97d976664c116b64041318cd922961e6df6038fbf3aa67b900f3372fd&expires_in=0&user_id=3377490

Соберем посты пользователей и их основную информацию <br> 
Ограничимся глубиной равной 1 - возьмем только друзей моих друзей

In [ ]:
# PERSON_LIST = []
# my_friend_list = VKApiConnector.get_user_friends_list(3377490)['items']
# PERSON_LIST.append(my_friend_list)
# for person in my_friend_list:
#     fr_list = VKApiConnector.get_user_friends_list(person)
#     if fr_list:
#         fr_list = fr_list['items']
#         PERSON_LIST.append(fr_list)
#     else:
#         continue    

In [ ]:
# PERSON_LIST_UNIQUE = [i for lst in PERSON_LIST for i in lst]
# PERSON_LIST_UNIQUE = list(set(PERSON_LIST_UNIQUE))
# print('cnt users', len(PERSON_LIST_UNIQUE))

# with open('data/PERSON_LIST_UNIQUE.pickle', 'wb') as handle:
#     pickle.dump(PERSON_LIST_UNIQUE, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('data/PERSON_LIST_UNIQUE.pickle', 'rb') as handle:
    PERSON_LIST_UNIQUE = pickle.load(handle)

Теперь для каждого пользователя выгрузим следующую информацию: name, surname, sex, posts. <br>
Попробуем найти зависимость между постами и полом человека

In [ ]:
VKApiConnector.get_user_info('20804, 34129, 410244251')

In [ ]:
# main_df = pd.DataFrame()
# def divide_chunks(l, n):       
#     # looping till length l 
#     for i in range(0, len(l), n):  
#         yield l[i:i + n]
# persons_chunks = list(divide_chunks(PERSON_LIST_UNIQUE, 999))

# for chunk in persons_chunks:
#     owners = ','.join(map(str,chunk))
#     owners_info = VKApiConnector.get_user_info(owners)
#     for person in owners_info:
#         if 'deactivated' in person.keys():
#             continue
#         if person['sex'] == 0:
#             continue
#         main_df = pd.concat([main_df, pd.DataFrame({'vk_id':[person['id']],
#                                                     'first_name':[person['first_name']],
#                                                     'last_name':[person['last_name']],
#                                                     'sex':[person['sex']]                                                   
#                                                    })])
# with open('data/main_df.pickle', 'wb') as handle:
#     pickle.dump(main_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data/main_df.pickle', 'rb') as handle:
    main_df = pickle.load(handle)
          

In [ ]:
main_df.head()

In [ ]:
N = main_df.shape[0]
N

In [ ]:
cnt = 0
user_post_df = pd.DataFrame()

for user_id in main_df['vk_id'].unique()[:200]:
    cnt +=1
    if cnt%100 == 0:
        print('Processing', cnt)
    response_wall = VKApiConnector.get_wall(user_id)
    if response_wall:
        print(user_id)
        count = response_wall['count']
        posts = response_wall['items']
        all_text = ''
        for item in posts:
            all_text += ' '+item['text']
        user_post_df = pd.concat([user_post_df, pd.DataFrame({'vk_id': [user_id],
                                                              'post_count': [count],
                                                              'post_text': [all_text]
                                                             })])


In [ ]:
main_df['vk_id'].unique()[:200]

In [6]:
#1835557
response = VKApiConnector.get_wall(1835557)

<Response [200]>


In [7]:
response.json()

{'error': {'error_code': 6,
  'error_msg': 'Too many requests per second',
  'request_params': [{'key': 'oauth', 'value': '1'},
   {'key': 'method', 'value': 'wall.get'},
   {'key': 'v', 'value': '5.68'},
   {'key': 'client_id', 'value': '6931114'},
   {'key': 'owner_id', 'value': '1835557'},
   {'key': 'count', 'value': '100'},
   {'key': 'filter', 'value': 'owner'}]}}

In [ ]:
with open('data/user_post_df.pickle', 'wb') as handle:
    pickle.dump(user_post_df, handle, protocol=pickle.HIGHEST_PROTOCOL)